<a href="https://colab.research.google.com/github/Douw450/2025DE245Prac2/blob/main/2025DE245Prac1_ExtractLoad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Python for Data Engineering

This notebook demonstrates how to read data from a public API, process it to create useful information, and store the result in an SQLite database.

It assumes you have read [this article](https://www.startdataengineering.com/post/python-for-de/#extract--load-read-and-write-data-to-any-system).

**Acknowledgment**: Parts of this notebook were developed with the assistance of ChatGPT, an AI language model by OpenAI, to support instructional design and coding tasks related to data engineering concepts.

In [13]:
# Install required packages
!pip install requests

In [14]:
# Import libraries
import requests
import sqlite3
from datetime import datetime
import os
os.remove("bitcoin_prices.db")


# Ask the user for the target currency
target_currency = input("Enter the target currency (e.g., zar, eur, gbp): ").lower()

Enter the target currency (e.g., zar, eur, gbp): zar


In [15]:
# Read data from CoinGecko API
url = f"https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd,{target_currency}"
response = requests.get(url)
data = response.json()

data
# Extract USD and target currency prices
usd_price = data["bitcoin"]["usd"]
target_price = data["bitcoin"][target_currency]

timestamp = datetime.now().isoformat()
print(f"Bitcoin price in USD: {usd_price}")
print(f"Bitcoin price in {target_currency.upper()}: {target_price}")

Bitcoin price in USD: 119993
Bitcoin price in ZAR: 2100441


We will extract the Bitcoin price in USD and convert it to ZAR using an estimated exchange rate.
This is an example of turning raw data into useful **information**.

In [16]:
# Function to convert USD to ZAR
def convert_usd_to_zar(usd_amount, rate=18.5):
    return round(usd_amount * rate, 2)

In [17]:
# Extract USD price and apply transformation
#usd_price = data["bitcoin"]["usd"]
#zar_price = convert_usd_to_zar(usd_price)
#timestamp = datetime.now().isoformat()

#print(f"Bitcoin price in USD: {usd_price}")
#print(f"Estimated price in ZAR: {zar_price}")


In [18]:
# Connect to SQLite database (or create it)
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Create table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS prices (
    timestamp TEXT PRIMARY KEY,
    usd REAL,
    target_currency TEXT,
    target_price REAL
)
''')

# Insert the new row
cursor.execute(
    "INSERT INTO prices (timestamp, usd, target_currency, target_price) VALUES (?, ?, ?, ?)",
    (timestamp, usd_price, target_currency, target_price)
)

# Commit changes and close connection
conn.commit()
conn.close()


In [19]:
# Check if data was successfully written
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Show the last 5 rows
rows = cursor.execute("SELECT * FROM prices ORDER BY timestamp DESC LIMIT 5").fetchall()
for row in rows:
    print(row)

conn.close()


('2025-08-13T09:26:47.654945', 119993.0, 'zar', 2100441.0)
